In [ ]:
import numpy as np
import csv

def load_titanic_dataset():
  with open('/content/drive/My Drive/Colab Notebooks/titanic_test.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader, None)
    rows = []
    for row in csvreader:
      rows.append(row)

  global data, input_cnt, output_cnt
  input_cnt, output_cnt = 30, 1
  data = np.zeros([len(rows), input_cnt+output_cnt])

  for n, row in enumerate(rows):
    if row[0] == 'male': data[n, 0] = 1
    if row[0] == 'female': data[n, 1] = 1
    data[n, 29:] = row[1:]

def arrange_data(mb_size):
  global data, shuffle_map, test_begin_idx
  shuffle_map = np.arange(data.shape[0])
  np.random.shuffle(shuffle_map)
  step_count = int(data.shape[0] * 0.8) // mb_size
  test_begin_idx = step_count * mb_size
  return step_count

def get_test_data():
  global data, shuffle_map, test_begin_idx, output_cnt
  test_data = data[shuffle_map[test_begin_idx:]]
  return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
  global data, shuffle_map, test_begin_idx, output_cnt
  if nth == 0:
    np.random.shuffle(shuffle_map[:test_begin_idx])
  train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
  return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

def eval_accuracy(output, y):
  mdiff = np.mean(np.abs((output - y)/y))
  return 1 - mdiff

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(30, 25)
    self.fc2 = nn.Linear(25, 20)
    self.fc3 = nn.Linear(20, 15)
    self.fc4 = nn.Linear(15, 5)
    self.fc5 = nn.Linear(5, 1)

  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.relu(x)
    x = self.fc5(x)
    return x

In [ ]:
# hyperparameters
LEARNING_RATE = 0.01 
epoch_count = 100
mb_size = 5

# dataset load
load_titanic_dataset()
step_count = arrange_data(mb_size)
test_x, test_y = get_test_data()

In [ ]:
# gpu check
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("device:", device)

# model set (gpu)
model = Net().to(device)
print(model)

# SGD optimizer
#optimizer = optim.SGD(model.parameters(), lr = LEARNING_RATE)
optimizer = optim.Adam(model.parameters())

# MSE loss
loss_fn = nn.MSELoss()

for epoch in range(epoch_count):
  losses, accs = [], []

  # set train mode
  model.train()

  for n in range(step_count):
    # optimizer init
    optimizer.zero_grad()

    # get train data
    train_x, train_y = get_train_data(mb_size, n)

    # dataset to torch
    x = torch.from_numpy(train_x).float()
    y = torch.from_numpy(train_y).float()
    x, y = x.to(device), y.to(device)

    # forward
    y_pred = model(x)
    acc = eval_accuracy(y_pred.to('cpu').detach().numpy(), train_y)
    accs.append(acc)

    # loss
    loss = loss_fn(y_pred, y)

    # backprop
    loss.backward()
    losses.append(loss.item())

    # weight, bias 업데이트
    optimizer.step()

  # run test & eval
  model.eval()
  x = torch.from_numpy(test_x).float()
  x = x.to(device)
  y_pred = model(x)
  acc = eval_accuracy(y_pred.to('cpu').detach().numpy(), test_y)

  print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
        format(epoch+1, np.mean(losses), np.mean(accs), acc))

device: cuda
Net(
  (fc1): Linear(in_features=30, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=15, bias=True)
  (fc4): Linear(in_features=15, out_features=5, bias=True)
  (fc5): Linear(in_features=5, out_features=1, bias=True)
)
Epoch 1: loss=4.060, accuracy=0.204/0.509
Epoch 2: loss=0.719, accuracy=0.533/0.536
Epoch 3: loss=0.665, accuracy=0.550/0.527
Epoch 4: loss=0.646, accuracy=0.557/0.551
Epoch 5: loss=0.630, accuracy=0.568/0.564
Epoch 6: loss=0.635, accuracy=0.569/0.566
Epoch 7: loss=0.628, accuracy=0.573/0.573
Epoch 8: loss=0.619, accuracy=0.575/0.573
Epoch 9: loss=0.626, accuracy=0.576/0.568
Epoch 10: loss=0.622, accuracy=0.578/0.563
Epoch 11: loss=0.622, accuracy=0.579/0.555
Epoch 12: loss=0.623, accuracy=0.576/0.571
Epoch 13: loss=0.617, accuracy=0.579/0.569
Epoch 14: loss=0.624, accuracy=0.576/0.568
Epoch 15: loss=0.620, accuracy=0.581/0.569
Epoch 16: loss=0.624, accuracy=0.578/0.576
Epoc